# Automatic import of Zurich fountain data to Wikidata
The following script downloads fountain data from Open Data Zurich, compares it to existing fountains in Wikidata for the same region, and creates Wikidata Quickstatement commands to complete the entries in Wikidata. New entities are created if no matching fountains are found.

## Initialize environment

In [1]:
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot

In [2]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
""").graph
endpoint=RemoteEndpoint(
   "http://query.wikidata.org/sparql"
   ,prefixes=prefixes
)

## Load data

In [3]:
data_url = 'https://data.stadt-zuerich.ch/dataset/brunnen/resource/d741cf9c-63be-495f-8c3e-9418168fcdbf/download/brunnen.json'

In [4]:
data_js = json.loads(urlopen(data_url).read())

### Rename columns to make them easier to work with

In [5]:
# convert to dataframe
df = pd.io.json.json_normalize(data_js['features'])
# extract coords
df['X'] = df['geometry.coordinates'].apply(lambda crds:crds[0])
df['Y'] = df['geometry.coordinates'].apply(lambda crds:crds[1])
# remove not needed columns
df = df.drop(columns=['geometry.coordinates', 'geometry.type', 'properties.objectid', 'type'])
# rename columns
df = df.rename(index=str, columns=
               {"properties.bezeichnung": "label_de", 
                "properties.brunnenart_txt": "fountain_type", 
                "properties.wasserart_txt": "water_type", 
                "properties.nummer":"operator_id",
                "properties.historisches_baujahr": "date"
               })

In [6]:
df.water_type.unique()

array(['Verteilnetz', 'Quellwasser', 'eigene Versorgung', 'Grundwasser'],
      dtype=object)

## Identify already existing fountains
### Query fountains from Wikidata

In [7]:
# Find the geographic extent of the data

buffer = 0.0003  # in degrees, corresponds to about 20-30 meters)
bounds = {
    'minX': df['X'].min() - buffer,
    'minY': df['Y'].min() - buffer,
    'maxX': df['X'].max() + buffer,
    'maxY': df['Y'].max() + buffer
}

In [8]:
# Query fountains (both water wells and fountains) from Wikidata within bounding box found above

query_string = """ SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{{
  # Enter coordinates
  SERVICE wikibase:box {{
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point({minX} {minY})"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point({maxX} {maxY})"^^geo:wktLiteral.
  }} .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q43483 }} || EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q483453 }}).
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  }} 
  OPTIONAL {{ ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}}
  OPTIONAL {{ ?place wdt:P571 ?date.}}
  OPTIONAL {{ ?place wdt:P5623 ?water_supply_type}}
  OPTIONAL {{ ?place wdt:P137 ?operator.}}
}}
  """.format(**bounds)

# Perform query
query_result = endpoint.select(query_string)

In [9]:
print(query_string)

 SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{
  # Enter coordinates
  SERVICE wikibase:box {
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point(8.45960259979614 47.3229261255644)"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point(8.61940272745742 47.431119712250506)"^^geo:wktLiteral.
  } .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS { ?place wdt:P31/wdt:P279* wd:Q43483 } || EXISTS { ?place wdt:P31/wdt:P279* wd:Q483453 }).
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  } 
  OPTIONAL { ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}
  OPTIONAL { ?place wdt:P571 ?date.}
  OPTIONAL { ?place wdt:P5623 ?water_supply_type}
  OPTIONAL { ?place wdt:P137 ?operator.}
}
  


### Tidy up data

In [10]:
# Extract coordinates from Wikidata results

query_result['X'] = query_result['location'].apply(lambda l:float(l.split('(')[1].split(' ')[0]))
query_result['Y'] = query_result['location'].apply(lambda l:float(l.split(' ')[1].split(')')[0]))

In [11]:
# remove duplicate entries

# duplicate entries are caused when e.g. a fountain has catalog codes from two catalogs
query_result = query_result.drop_duplicates('place')

### Compute distances between fountains

In [12]:
# helper function to compute distances on the globe, returns distances in meters
def spherical_dist(pos1, pos2, r=6371000):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))


# compute distances from each ODZ fountain to each Wikidata fountain
distances = spherical_dist(df[['X','Y']].values[:, None], query_result[['X','Y']].values)

### Identify nearest and second nearest matches for each ODZ fountain

In [13]:
# indexes of nearest fountains
nearest_idx = np.argmin(distances, axis=1).tolist()

# QID of nearest fountains
df['nearest_qid'] = query_result.iloc[nearest_idx]['place'].apply(lambda id:id[3:]).tolist()

# distance to nearest fountain
df['nearest_distance'] = np.min(distances, axis=1).tolist()


# then remove nearest
i_line=0
for i_col in nearest_idx:
    distances[i_line, i_col] = 100000
    i_line += 1
# find distance to second nearest
df['2nd_nearest_distance'] = np.min(distances, axis=1).tolist()

df.head(100)

,label_de,fountain_type,date,operator_id,water_type,X,Y,nearest_qid,nearest_distance,2nd_nearest_distance
0,Aussichtsturm,öffentlicher Brunnen,1970.0,510,Verteilnetz,8.599255,47.369752,Q55165903,0.000000,342.913025
1,None,öffentlicher Brunnen,1933.0,349,Verteilnetz,8.590811,47.369293,Q55165905,0.000000,541.765153
2,Biberlinterrasse,öffentlicher Brunnen,1965.0,365,Quellwasser,8.575754,47.366130,Q55165919,0.000000,360.590819
3,None,öffentlicher Brunnen,1910.0,338,Quellwasser,8.564845,47.370993,Q27230192,1.980033,190.081520
4,None,Notwasserbrunnen,1988.0,6069,Quellwasser,8.564390,47.369327,Q55165936,0.000000,188.767471
5,None,öffentlicher Brunnen,1903.0,317,Quellwasser,8.560110,47.365197,Q55165939,0.000000,179.860503
6,None,öffentlicher Brunnen,1907.0,329,Quellwasser,8.562215,47.365124,Q55165953,0.000000,213.801415
7,None,öffentlicher Brunnen,1906.0,327,Quellwasser,8.560303,47.366987,Q55166016,0.000000,156.849604
8,None,öffentlicher Brunnen,1903.0,315,Quellwasser,8.556960,47.368494,Q55166017,0.000000,133.190434
9,None,öffentlicher Brunnen,1892.0,313,Quellwasser,8.555891,47.369544,Q55166032,0.000000,15.021392


### Find out what information already exists for the nearest fountains

In [14]:
# does nearest have label in german?
df['nearest_has_label_de'] = (query_result.iloc[nearest_idx]['place'].apply(lambda p:p[3:]) != query_result.iloc[nearest_idx]['placeLabel']).tolist()

# does nearest have date?
df['nearest_has_date'] = query_result.iloc[nearest_idx]['date'].apply(lambda d:d is not None).tolist()

# does nearest have operator?
df['nearest_has_operator'] = query_result.iloc[nearest_idx]['operator'].apply(lambda id:id is not None).tolist()

# does nearest have catalog code?
df['nearest_has_code'] = query_result.iloc[nearest_idx]['catalog_code'].apply(lambda id:id is not None).tolist()

# does nearest have water type?
df['nearest_has_water_type'] = query_result.iloc[nearest_idx]['water_supply_type'].apply(lambda id:id is not None).tolist()

### Decide on whether nearest fountain should be considered a match

In [15]:
# The nearest fountain is a match if: 
# - no further than x m away
# - 2nd nearest fountain at nearest least ratio_min further away than the nearest fountain
def validate_proposal(qid, d1, d2, dmax=10, ratio_min=0.5):
    
    if d1 == 0 or (d1<=dmax and d2/d1-1 >= ratio_min):
        return 'match'
    elif d1<=dmax and d2/d1-1 < ratio_min:
        return 'unclear'
    else:
        return 'no match'
    
for index, row in df.iterrows():
    df.loc[index, 'match_found'] = validate_proposal(
        row['nearest_qid'], 
        row['nearest_distance'], 
        row['2nd_nearest_distance'],
        dmax=15
    )
dffinal = df.drop(columns=['nearest_distance', '2nd_nearest_distance'])

In [16]:
dffinal.head()

,label_de,fountain_type,date,operator_id,water_type,X,Y,nearest_qid,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,Aussichtsturm,öffentlicher Brunnen,1970.0,510,Verteilnetz,8.599255,47.369752,Q55165903,True,True,True,True,False,match
1,None,öffentlicher Brunnen,1933.0,349,Verteilnetz,8.590811,47.369293,Q55165905,False,True,True,True,False,match
2,Biberlinterrasse,öffentlicher Brunnen,1965.0,365,Quellwasser,8.575754,47.366130,Q55165919,True,True,True,True,False,match
3,None,öffentlicher Brunnen,1910.0,338,Quellwasser,8.564845,47.370993,Q27230192,True,True,True,True,False,match
4,None,Notwasserbrunnen,1988.0,6069,Quellwasser,8.564390,47.369327,Q55165936,False,True,True,True,False,match


## Create Quickstatement commands from data
### Helper functions to format content according to Quickstatements v1 syntax

In [19]:
def process_coordinates(x, y):
    # format geographic coordinates
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)


def process_year(date):
    # format date
    if np.isnan(date):
        return ''
    else:
        return '+{0:4d}-00-00T00:00:00Z/9'.format(int(date))

    
fountain_type_map = {
    'öffentlicher Brunnen': 'Q53628296',
    'Notwasserbrunnen': 'Q53628522',
    'privater Brunnen': 'Q53629707',
    'Brunnen in städtischer Liegenschaft': 'Q53628618',
    'Brunnen des Verschönerungsvereins': 'Q53628761',
    'Brunnen mit eigener Versorgung': 'Q53630002'
}

water_type_map = {
    'Verteilnetz': 'Q53633635',
    'Quellwasser': 'Q1881858',
    'eigene Versorgung': 'Q53634173',
    'Grundwasser': 'Q161598'
}

def process_fountain_type(type):
    # translate fountain types to wikidata values
    return fountain_type_map[type]


def process_water_type(type):
    # translate water types to wikidata values
    return water_type_map[type]


def process_label_de(text):
    # process German language labels
    if text is None:
        return ''
    elif 'brunnen' in text.lower():
        return '"{}"'.format(text)
    else:
        return '"Brunnen ({})"'.format(text)
    

def createline(lines, item, prop, value, qualifiers=[]):
    # general function to create Quickstatement v1 commands
    if value != '' and value != '""':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += '\n'
        lines.append(statement)
    return lines

### Create statements, taking care not to overwrite existing data

In [20]:
# initialize command storage list
lines = []

for index, row in dffinal.iterrows():
    
    # either create new or edit existing entity
    if row['match_found'] == 'no match':
        # create a new fountain
        lines.append('CREATE\n')
        item = 'LAST'
    elif row['match_found'] == 'unclear':
        print('unclear match')
        print(row)
        continue
    elif row['match_found'] == 'match':
        # update existing fountain
        item = row['nearest_qid']
        
        
    # Add this basic information only if creating a new entity
    if item == 'LAST':
        # instance of drinking fountain
        lines = createline(lines, item, 'P31', 'Q1630622')

        # instance of specific water fountain type
        lines = createline(lines, item, 'P31', process_fountain_type(row['fountain_type']))

        # coordinates
        lines = createline(lines, item, 'P625', process_coordinates(row['X'], row['Y']))
        
        
    # For other properties, add information if the entity is new or if property does not yet exist
    
    # label in german
    if item == 'LAST' or not row['nearest_has_label_de']:
        lines = createline(lines, item, 'Lde', process_label_de(row['label_de']))
    
    # creation date
    if item == 'LAST' or not row['nearest_has_date']:
        lines = createline(lines, item, 'P571', process_year(row['date']))

    # operated by WVZ    
    if item == 'LAST' or not row['nearest_has_operator']:
        lines = createline(lines, item, 'P137', 'Q27229237')
        
    
    # operated by WVZ    
    if item == 'LAST' or not row['nearest_has_water_type']:
        lines = createline(lines, item, 'P5623', process_water_type(row['water_type']))
    
    
    # catalog number can always be added (it is hard to check for)
    lines = createline(lines, item, 'P528', '"{}"'.format(row['operator_id']), [{
        'prop': 'P972',
        'value': 'Q53629101'
    }])

# Write commands to file

In [21]:
with io.open("quickstatement_commands.txt", "w", encoding='utf8') as f:
    f.writelines(lines)